In [2]:

import re
import pandas as pd

In [3]:

def parse_duty_value(x):
    """
    - "Rs. 42 / kg" or "₹42/kg"    -> (42.0, "INR/kg")
    - "120/kg"                     -> (120.0, "INR/kg")
    - plain number "10"            -> (0.10, "percentage")
    - numeric 10 or 10.0           -> (0.10, "percentage")
    - NaN/empty                    -> (None, None)
    """
    if pd.isna(x):
        return None, None

    # Numeric → percentage
    if isinstance(x, (int, float)):
        return float(x) / 100.0, "percentage"

    s = str(x).strip()

    # 1) Currency-per-unit pattern
    m = re.match(r'(?:Rs\.?|₹)?\s*([\d\.]+)\s*/\s*(\w+)', s, flags=re.IGNORECASE)
    if m:
        value = float(m.group(1))
        unit  = m.group(2)
        return value, f"INR/{unit}"

    # 2) Pure number → percentage
    if re.fullmatch(r'[\d\.]+', s):
        return float(s) / 100.0, "percentage"

    return None, None

def format_duty(value, unit):
    if value is None or pd.isna(value):
        return ""
    if unit == "percentage":
        return f"{value * 100:.2f}%"
    else:
        # e.g. unit == "INR/kg"
        return f"{value:.2f} {unit}"

def normalize_and_prepare_display(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy().applymap(lambda x: x.strip() if isinstance(x, str) else x)
    duty_cols = [
        'Basic Duty (SCH)',
        'Basic Duty (NTFN)',
        'Specific Duty (Rs)',
        'IGST',
        '10% SWS',
        'Total duty with SWS of 10% on BCD',
        'Total Duty Specific',
        'Pref. Duty (A)',
    ]

    for col in duty_cols:
        if col not in df:
            continue
        parsed = df[col].apply(parse_duty_value)
        df[f'{col}_value'], df[f'{col}_unit'] = zip(*parsed)
        df[f'{col}_display'] = [
            format_duty(v, u) for v, u in zip(df[f'{col}_value'], df[f'{col}_unit'])
        ]

    return df

# Example:
# df_raw = pd.read_excel('chapter.xlsx')
# df_out = normalize_and_prepare_display(df_raw)
# df_out[['IGST_display','Specific Duty (Rs)_display']]
#   might show: ["10.00%",   "42.00 INR/kg"]


In [4]:


def extract_policy_links(cell: str):
    """
    From a string like "Restricted*1,2" or "Free*" or "Conditional2":
      - returns ("Restricted", ["*","1","2"])
      - returns ("Free", ["*"])
      - returns ("Conditional", ["2"])
    If no trailing markers, returns (original_text, []).
    """
    if pd.isna(cell) or not isinstance(cell, str):
        return None, []

    s = cell.strip()
    # split off trailing sequence of *, digits and commas
    m = re.match(r'^(?P<text>.*?)(?P<refs>[\*\d,]+)$', s)
    if not m:
        return s, []

    base, refs = m.group("text").strip(), m.group("refs")
    # normalize refs into list of individual tokens
    tokens = []
    for part in refs.split(','):
        part = part.strip()
        if not part:
            continue
        # if it's multiple asterisks, keep each
        if set(part) == {"*"}:
            tokens += ["*"] * len(part)
        else:
            # each character that is a digit
            tokens += list(part)
    return base, tokens

def attach_policy_links(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for col in ("Import Policy", "Export Policy"):
        if col not in df:
            continue

        extracted = df[col].apply(extract_policy_links)
        df[f"{col}_text"], df[f"{col}_note_refs"] = zip(*extracted)

    return df




In [14]:

# Step 1: your existing compute_level (for Tariff rows only)
def seed_base_level(row):
    lvl = row.get("Level")
    desc = str(row.get("Item Description","")).strip().lower()
    # Chapter headers always level = 0
    if desc.startswith("chapter"):
        return 0
    # hyphens → level = count of hyphens
    if isinstance(lvl, str) and re.fullmatch(r"-+", lvl.strip()):
        return len(lvl.strip())
    # otherwise leave as None (so we don’t accidentally force zeros)
    return None



In [16]:
def main_excel_read(input_file: str):
  # Read the Excel file
  df = pd.read_excel(input_file)
  df.columns = (
  df.columns
    .str.strip()                      # remove leading/trailing spaces
    .str.replace(r"\s+", " ", regex=True)  # collapse inner whitespace
  )

  # Normalize and prepare display
  df = normalize_and_prepare_display(df)
  # Attach policy links
  df = attach_policy_links(df) 
  # apply it
  df["level"] = df.apply(seed_base_level, axis=1)

  # 2) Now override only the Tariff rows
  mask = df["Remark"] == "Tariff"

  # a) 4-digit HSN → level 1
  mask_4d = mask & df["HS Code"].astype(str).str.fullmatch(r"\d{4}")
  df.loc[mask_4d, "level"] = 1

  # b) hyphens in Level → level = hyphens count + 1
  def hyphens_plus_one(s):
    return len(s.strip()) + 1
  mask_hyph = mask & df["Level"].astype(str).str.fullmatch(r"-+")
  df.loc[mask_hyph, "level"] = df.loc[mask_hyph, "Level"].apply(hyphens_plus_one)

  # c) any other Tariff row (no HSN-4 or hyphens) → fallback level 1
  mask_other = mask & ~mask_4d & ~mask_hyph
  df.loc[mask_other, "level"] = 1


  df.to_excel(f"{input_file}_with_links.xlsx", index=False)

In [17]:
main_excel_read("Chapter 25.xlsx")

/var/folders/07/y36f8rx151z3xys0mnl4p0y40000gn/T/ipykernel_49654/3302561760.py:41: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.copy().applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [4]:
# Hierarchical JSON Generator (Jupyter Notebook)
import pandas as pd
import json

def build_hierarchy(
    df,
    level_col='level',
    remark_col='Remark',
    note_text_cols=None,
    hs_code_col='HS Code'
):
    """
    Build nested hierarchy:
    - Normalize HS Code by stripping all spaces.
    - Node rows: any row where remark_col != 'Notes'. level_col gives depth (NaN → 0).
    - Note rows (remark_col == 'Notes'): attach to most recent level-1 node if exists, otherwise level-0.
    - Discontinuous levels attach under nearest existing ancestor < current level.
    """
    # Clean headers and normalize HS Code
    df = df.rename(columns=lambda c: c.strip())
    df[hs_code_col] = df[hs_code_col].apply(
        lambda x: str(x).replace(' ', '') if pd.notna(x) else x
    )
    
    if note_text_cols is None:
        note_text_cols = ['Item Description']

    root = []
    level_nodes = {}   # depth -> last node at that depth

    for _, row in df.iterrows():
        remark = str(row.get(remark_col)).strip()

        if remark != 'Notes':
            # Node row
            raw_lvl = row.get(level_col)
            try:
                depth = int(raw_lvl) if pd.notna(raw_lvl) else 0
            except:
                depth = 0

            # Copy all columns and add notes/children
            node = {col: (None if pd.isna(val) else val) for col, val in row.items()}
            node['notes'] = []
            node['children'] = []

            # Attach node
            if depth == 0:
                root.append(node)
            else:
                ancestors = [d for d in level_nodes if d < depth]
                if ancestors:
                    parent = level_nodes[max(ancestors)]
                    parent['children'].append(node)
                else:
                    root.append(node)

            level_nodes[depth] = node

        else:
            # Note row: always attach to level-1 if present, else level-0
            if 1 in level_nodes:
                target = level_nodes[1]
            elif 0 in level_nodes:
                target = level_nodes[0]
            else:
                continue

            for col in note_text_cols:
                text = row.get(col)
                if pd.notna(text):
                    target['notes'].append(str(text).strip())

    return root

# ─── USER SETTINGS ─────────────────────────────────────────────
input_path      = 'Chapter25_clean.xlsx'    # your Excel file
output_path     = 'hierarchy_ver3.json'    # desired JSON output
level_col       = 'level'             # column for depth
remark_col      = 'Remark'           # column marking 'Tariff'/'Notes'/'Chapter'
note_text_cols  = [
    'Item Description',
    'Import Policy_text',
    'Export Policy_text'
]
hs_code_col     = 'HS Code'          # column to normalize (remove spaces)
# ────────────────────────────────────────────────────────────────

# Load data, build hierarchy, and save JSON

df = pd.read_excel(input_path)
hierarchy = build_hierarchy(
    df,
    level_col=level_col,
    remark_col=remark_col,
    note_text_cols=note_text_cols,
    hs_code_col=hs_code_col
)

with open(output_path, 'w', encoding='utf-8') as fp:
    json.dump(hierarchy, fp, indent=2, ensure_ascii=False)

print(f"Saved hierarchical JSON with {len(hierarchy)} top-level nodes to {output_path}")

Saved hierarchical JSON with 1 top-level nodes to hierarchy_ver3.json
